# Distance errors in APOGEE DR17

In [5]:
import matplotlib.pyplot as plt
from apogee_tools import import_apogee
import paths
from utils import quad_add

In [7]:
apogee_data = import_apogee()
apogee_data['GAIAEDR3_R_PHOTOGEO_ERR'] = quad_add(
    apogee_data['GAIAEDR3_R_HI_PHOTOGEO'] - apogee_data['GAIAEDR3_R_MED_PHOTOGEO'], 
    apogee_data['GAIAEDR3_R_MED_PHOTOGEO'] - apogee_data['GAIAEDR3_R_LO_PHOTOGEO'])

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(9, 4), 
                        sharex=True, sharey=True)
plt.style.use(paths.styles / 'presentation.mplstyle')
xlim = (0, 24)
ylim = (-12, 12)
extent = [xlim[0], xlim[1], ylim[0], ylim[1]]
# MWM
norm = LogNorm()
hb0 = axs[0].hexbin(apogee_data['GALR'], apogee_data['GALZ'], 
                    C=apogee_data['GAIAEDR3_R_PHOTOGEO_ERR'], reduce_C_function=np.median,
                    gridsize=50, cmap='inferno', norm=norm, 
                    extent=extent, linewidths=0.2)
axs[0].set_title('Median distance error')
axs[0].set_aspect('equal')
axs[0].xaxis.set_minor_locator(MultipleLocator(2))
axs[0].yaxis.set_major_locator(MultipleLocator(10))
axs[0].yaxis.set_minor_locator(MultipleLocator(2))
# APOGEE
hb1 = axs[1].hexbin(apogee_data['GALR'], apogee_data['GALZ'], 
                    gridsize=50, cmap='inferno', norm=norm, 
                    extent=extent, linewidths=0.2)
axs[1].set_aspect('equal')
axs[1].set_title('APOGEE DR17')
plt.subplots_adjust(right=0.88, wspace=0.05)
cax = plt.axes([0.9, 0.11, 0.02, 0.77])
fig.colorbar(hb1, cax, label='Number of stars')
# Common axis labels
fig.add_subplot(111, frameon=False)
plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
plt.xlabel('Galactocentric radius [kpc]')
plt.ylabel('Midplane distance [kpc]')
plt.show()